In [1]:
import os
import ast
import numpy as np
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
# Setup
csv_path = '/kaggle/input/magic-the-gathering-trimmed-dataset/mtg-trimmed-3.csv'
pd.set_option("display.max_columns", None)

In [3]:
# Load the data and take a look
df = pd.read_csv(csv_path)
print(df.shape)
df.head(3)

(61644, 24)


,released_at,prices,rarity,type_line,color_identity,power,toughness,produced_mana,cmc,hand_modifier,life_modifier,artist,games,reprint,set,set_type,booster,border_color,collector_number,digital,full_art,illustration_id,promo_types,story_spotlight
0,2006-10-06,"{'usd': '0.92', 'usd_foil': '6.35', 'eur': '0....",uncommon,Creature — Sliver,['R'],3,3,NaN,6.0,NaN,NaN,Paolo Parente,"['paper', 'mtgo']",False,tsp,expansion,True,black,157,False,False,2fcca987-364c-4738-a75b-099d8a26d614,NaN,False
1,2009-10-02,"{'usd': '0.22', 'usd_foil': '2.46', 'eur': '0....",common,Creature — Kor Soldier,['W'],2,2,NaN,2.0,NaN,NaN,Kieran Yanner,"['paper', 'mtgo']",False,zen,expansion,True,black,21,False,False,de0310d1-e97f-46e0-bc16-c980c2adedee,NaN,False
2,2015-05-22,"{'usd': '0.09', 'usd_foil': None, 'eur': None,...",common,Token Creature — Spirit,['W'],1,1,NaN,0.0,NaN,NaN,Mike Sass,['paper'],True,tmm2,token,True,black,5,False,False,1dbe0618-dd47-442c-acf6-ac5e4b136e5a,['setpromo'],False


### Convert string representations of dicts (prices column) to actual dictionaries to build a DataFrame and concatenate it with original

In [4]:
prices = list(df.prices)

def convert_str_to_dict(strings_list):
    return [ast.literal_eval(x) for x in strings_list]

prices_fix = convert_str_to_dict(prices)
prices_df = pd.DataFrame(prices_fix)

# Just to make sure both DataFrames are aligned
df.reset_index(drop=True, inplace=True)
prices_df.reset_index(drop=True, inplace=True)

print(f"{df.shape} - Original")
print(f"{prices_df.shape} - DataFrame created from 'prices' column")

(61644, 24) - Original
(61644, 5) - DataFrame created from 'prices' column


In [5]:
dataframes = [df, prices_df]
df = pd.concat(dataframes, axis=1)
df.head(1)

,released_at,prices,rarity,type_line,color_identity,power,toughness,produced_mana,cmc,hand_modifier,life_modifier,artist,games,reprint,set,set_type,booster,border_color,collector_number,digital,full_art,illustration_id,promo_types,story_spotlight,usd,usd_foil,eur,eur_foil,tix
0,2006-10-06,"{'usd': '0.92', 'usd_foil': '6.35', 'eur': '0....",uncommon,Creature — Sliver,['R'],3,3,NaN,6.0,NaN,NaN,Paolo Parente,"['paper', 'mtgo']",False,tsp,expansion,True,black,157,False,False,2fcca987-364c-4738-a75b-099d8a26d614,NaN,False,0.92,6.35,0.20,1.00,0.03


#### Remove and rename a few columns and stick to one currency

In [6]:
df.drop(columns=['prices', 'eur', 'eur_foil', 'tix', 'collector_number', 'digital'], inplace=True)
df.rename(columns={"rarity": "rare", "released_at": "released", "color_identity": "color", "type_line": "type"}, inplace=True)

### Map numbers to values, clean up a little and change dtypes

In [7]:
# Remove unnecessary characters
df['games'] = df['games'].apply(lambda x: x.strip("['']").replace("'", ""))
df['color'] = df['color'].apply(lambda x: x.strip("['']").replace("'", ""))

# There are some unwanted characters here, let's get rid of them and only keep digits
pattern = r'([0-9]*[.]?[0-9]+)'
df['power'] = df['power'].str.extract(pattern)
df['toughness'] = df['toughness'].str.extract(pattern)

# Converting to categories (astype='category') gives an error when fitting the model (it expects numerical values), so I go with following approach
# Mapping colors
colors = list(df['color'].unique())
nums = [i for i in range(1,33)]
color_mapping = dict(zip(colors, nums))

df['color'] = df['color'].map(color_mapping)

# Mapping rarities
rare = list(df.rare.unique())
nums_2 = [i for i in range(1,7)]
rare_mapping = dict(zip(rare, nums_2))

df['rare'] = df['rare'].map(rare_mapping)

# Mapping set type
set_type = list(df.set_type.unique())
nums_3 = [i for i in range(1,21)]
set_type_mapping = dict(zip(set_type, nums_3))

df['set_type'] = df['set_type'].map(set_type_mapping)

# Mapping mana and modifiers to 0 (if NaN) and 1 (if anything else)
df.loc[df['produced_mana'].isna() == False, 'produced_mana'] = 1
df.loc[df['produced_mana'].isna() == True, 'produced_mana'] = 0

df.loc[df['hand_modifier'].isna() == False, 'hand_modifier'] = 1
df.loc[df['hand_modifier'].isna() == True, 'hand_modifier'] = 0

df.loc[df['life_modifier'].isna() == False, 'life_modifier'] = 1
df.loc[df['life_modifier'].isna() == True, 'life_modifier'] = 0

# Mapping boolean values to 0 and 1
def convert_bool(row):
    """Return int value of a bool"""
    return int(row == True)

df['reprint'] = df['reprint'].apply(lambda x: convert_bool(x))
df['booster'] = df['booster'].apply(lambda x: convert_bool(x))
df['full_art'] = df['full_art'].apply(lambda x: convert_bool(x))
df['story_spotlight'] = df['story_spotlight'].apply(lambda x: convert_bool(x))

# Prepare "type" column for the next step (extract main type, like "Creature", "Enchantment" etc.)
def make_list(row):
    """Split a string"""
    return row.replace('—', ',').split(',')

df['type'] = df['type'].apply(lambda x: make_list(x))
df['type'] = df['type'].apply(lambda x: [x[0].strip()])

# Adjust dtypes
df['released'] = pd.to_datetime(df['released'], format='%Y-%m-%d')
df[['usd', 'usd_foil', 'power', 'toughness']] = df[['usd', 'usd_foil', 'power', 'toughness']].astype(float)
df[['cmc', 'hand_modifier', 'life_modifier', 'produced_mana']] = df[['cmc', 'hand_modifier', 'life_modifier', 'produced_mana']].astype(int)
# df[['rare', 'color', 'set_type']] = df[['rare', 'color', 'set_type']].astype('category')

In [8]:
df.head()

,released,rare,type,color,power,toughness,produced_mana,cmc,hand_modifier,life_modifier,artist,games,reprint,set,set_type,booster,border_color,full_art,illustration_id,promo_types,story_spotlight,usd,usd_foil
0,2006-10-06,1,[Creature],1,3.0,3.0,0,6,0,0,Paolo Parente,"paper, mtgo",0,tsp,1,1,black,0,2fcca987-364c-4738-a75b-099d8a26d614,NaN,0,0.92,6.35
1,2009-10-02,2,[Creature],2,2.0,2.0,0,2,0,0,Kieran Yanner,"paper, mtgo",0,zen,1,1,black,0,de0310d1-e97f-46e0-bc16-c980c2adedee,NaN,0,0.22,2.46
2,2015-05-22,2,[Token Creature],2,1.0,1.0,0,0,0,0,Mike Sass,paper,1,tmm2,2,1,black,0,1dbe0618-dd47-442c-acf6-ac5e4b136e5a,['setpromo'],0,0.09,NaN
3,2017-09-29,2,[Creature],3,1.0,2.0,0,3,0,0,Chris Rallis,"arena, paper, mtgo",0,xln,1,1,black,0,e0a40a54-9216-4c86-b9e3-daed04abc310,NaN,0,0.03,0.24
4,1994-04-01,3,[Enchantment],4,NaN,NaN,0,1,0,0,Rob Alexander,paper,1,3ed,3,1,white,0,1eac4d23-2b24-4f4a-b73f-25607c13b806,NaN,0,1.64,NaN


### Explode the categories in "type" column to it's own DataFrame

In [9]:
mlb = MultiLabelBinarizer()
encoded = pd.DataFrame(mlb.fit_transform(df['type']), columns=mlb.classes_, index=df.index)
encoded.drop(columns=['instant'], inplace=True)
encoded.head(1)

,Artifact,Artifact // Artifact,Artifact // Creature,Artifact // Land,Artifact // Sorcery,Artifact Creature,Artifact Land,Basic Land,Basic Snow Land,Card,Card // Card,Card // Token Creature,Conspiracy,Creature,Dungeon,Eaturecray,Elemental Instant,Emblem,Enchantment,Enchantment // Creature,Enchantment Creature,Enchantment Land,Hero,Hero Artifact,Host Artifact Creature,Host Creature,Instant,Instant // Instant,Instant // Land,Instant // Sorcery,Instant // Sorcery // Sorcery,Instant Creature,Land,Land // Land,Land // Legendary Creature,Land Creature,Legendary Artifact,Legendary Artifact // Legendary Land,Legendary Artifact Creature,Legendary Creature,Legendary Enchantment,Legendary Enchantment // Legendary Land,Legendary Enchantment Artifact,Legendary Enchantment Creature,Legendary Land,Legendary Planeswalker,Legendary Snow Creature,Legendary Snow Enchantment,Legendary Snow Land,Legendary Sorcery,Legendary Token Creature,Ongoing Scheme,Phenomenon,Plane,Planeswalker,Scariest Creature You’ll Ever See,Scheme,Snow Artifact,Snow Artifact Creature,Snow Creature,Snow Enchantment,Snow Instant,Snow Land,Snow Sorcery,Sorcery,Sorcery // Creature,Sorcery // Instant,Sorcery // Land,Sorcery // Sorcery,Summon,Summon Dragon,Summon Goblin,Summon Jaguar,Summon Knights,Summon Specter,Summon Wolf,Token,Token // Token Creature,Token Artifact,Token Artifact Creature,Token Creature,Token Elite Creature,Token Enchantment,Token Enchantment Artifact Creature,Token Enchantment Creature,Token Legendary Artifact,Token Legendary Creature,Token Legendary Enchantment Creature,Token Snow Artifact,Tribal Artifact,Tribal Enchantment,Tribal Instant,Tribal Sorcery,Vanguard,World Enchantment
0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### Let's work on these columns

In [10]:
# Columns to extract information from
artifacts = encoded.columns[0:7]
basic = encoded.columns[7:9]
card = encoded.columns[9:12]
enchantment = encoded.columns[18:22]
hero = encoded.columns[22:24]
host = encoded.columns[24:26]
instant = encoded.columns[26:32]
land = encoded.columns[32:36]
legend = encoded.columns[36:51]
snow = encoded.columns[57:64]
sorcery = encoded.columns[64:69]
summon = encoded.columns[69:76]
token = encoded.columns[76:89]
tribal = encoded.columns[89:93]

cats = [artifacts, basic, card, enchantment, hero, host, instant, land, legend, snow, sorcery, summon, token, tribal]

# Prepare to drop all these columns
cols_to_drop =[]
for cat in cats:
    for x in cat:
        cols_to_drop.append(x)

In [11]:
def extract_one(x):
    if '1' in x:
        return 1
    return 0

for cat in cats:
    encoded[str(cat)[8:12]] = encoded[cat].apply(lambda x: extract_one(','.join(x.astype(str))), axis=1)
    
# Cleanup
encoded.drop(columns=(cols_to_drop), inplace=True)
encoded.rename(columns={"Scariest Creature You’ll Ever See": "Scary Creature"}, inplace=True)

reverse_encoded = encoded.idxmax(axis=1)
reverse_encoded.head()

0    Creature
1    Creature
2        Toke
3    Creature
4        Ench
dtype: object

#### Now add it to the main DataFrame, reorder columns, drop what we don't need, rename and change dtype

In [12]:
df['type_new'] = reverse_encoded

cols = df.columns.tolist()
cols = cols[-3:-1] + cols[0:2] + [cols[-1]] + cols[2:-3]
df = df[cols]

df.drop(columns=['type', 'set'], inplace=True)
df.rename(columns={'type_new': 'type'}, inplace=True)
# df['type'] = df['type'].astype('category')

types = list(df['type'].unique())
nums_4 = [i for i in range(1,26)]
types_mapping = dict(zip(types, nums_4))

df['type'] = df['type'].map(types_mapping)

In [13]:
df.head()

,usd,usd_foil,released,rare,type,color,power,toughness,produced_mana,cmc,hand_modifier,life_modifier,artist,games,reprint,set_type,booster,border_color,full_art,illustration_id,promo_types,story_spotlight
0,0.92,6.35,2006-10-06,1,1,1,3.0,3.0,0,6,0,0,Paolo Parente,"paper, mtgo",0,1,1,black,0,2fcca987-364c-4738-a75b-099d8a26d614,NaN,0
1,0.22,2.46,2009-10-02,2,1,2,2.0,2.0,0,2,0,0,Kieran Yanner,"paper, mtgo",0,1,1,black,0,de0310d1-e97f-46e0-bc16-c980c2adedee,NaN,0
2,0.09,NaN,2015-05-22,2,2,2,1.0,1.0,0,0,0,0,Mike Sass,paper,1,2,1,black,0,1dbe0618-dd47-442c-acf6-ac5e4b136e5a,['setpromo'],0
3,0.03,0.24,2017-09-29,2,1,3,1.0,2.0,0,3,0,0,Chris Rallis,"arena, paper, mtgo",0,1,1,black,0,e0a40a54-9216-4c86-b9e3-daed04abc310,NaN,0
4,1.64,NaN,1994-04-01,3,3,4,NaN,NaN,0,1,0,0,Rob Alexander,paper,1,3,1,white,0,1eac4d23-2b24-4f4a-b73f-25607c13b806,NaN,0


### Let's take a look at the prices.<br>For the sake of this experiment, I'll fill the missing values in "usd" with the mean of "usd_foil" and keep only "usd" column.

In [14]:
print(f"Number of missing values in 'usd' column = {df.usd.isna().sum()}")
print(f"Mean value of 'usd' column = {df.usd.mean():.2f}")
print(f"Mean value of 'usd_foil' column = {df.usd_foil.mean():.2f}")

Number of missing values in 'usd' column = 12902
Mean value of 'usd' column = 9.92
Mean value of 'usd_foil' column = 9.73


In [15]:
df['usd'].fillna(df.usd_foil.mean(), inplace=True)
df.drop(columns='usd_foil', inplace=True)

#### We can see that the mean value of the price doesn't change too much after filling nans

In [16]:
print(f"Mean value of 'usd' column after filling the missing values = {df.usd.mean():.2f}")

Mean value of 'usd' column after filling the missing values = 9.88


### I'll keep missing values in "power" and "toughness" columns for now and I won't use them as features. When I learn more about the game, I should have a better idea how to deal with them<br>There are still some untreated columns, namely: "illustration_id", "artist", "promo_types", "border_color" and "games"

#### That's what I have so far

In [17]:
df.head()

,usd,released,rare,type,color,power,toughness,produced_mana,cmc,hand_modifier,life_modifier,artist,games,reprint,set_type,booster,border_color,full_art,illustration_id,promo_types,story_spotlight
0,0.92,2006-10-06,1,1,1,3.0,3.0,0,6,0,0,Paolo Parente,"paper, mtgo",0,1,1,black,0,2fcca987-364c-4738-a75b-099d8a26d614,NaN,0
1,0.22,2009-10-02,2,1,2,2.0,2.0,0,2,0,0,Kieran Yanner,"paper, mtgo",0,1,1,black,0,de0310d1-e97f-46e0-bc16-c980c2adedee,NaN,0
2,0.09,2015-05-22,2,2,2,1.0,1.0,0,0,0,0,Mike Sass,paper,1,2,1,black,0,1dbe0618-dd47-442c-acf6-ac5e4b136e5a,['setpromo'],0
3,0.03,2017-09-29,2,1,3,1.0,2.0,0,3,0,0,Chris Rallis,"arena, paper, mtgo",0,1,1,black,0,e0a40a54-9216-4c86-b9e3-daed04abc310,NaN,0
4,1.64,1994-04-01,3,3,4,NaN,NaN,0,1,0,0,Rob Alexander,paper,1,3,1,white,0,1eac4d23-2b24-4f4a-b73f-25607c13b806,NaN,0
